# 26章　コードチューニングテクニック
- この章では、速度の改善を主に扱う。コードを小さくする方法も一部扱う

## コードチューニングとリファクタリングの違い
- コードチューニング:内部構造を改悪する代わりに、パフォーマンスをえること
- リファクタリング:振る舞いを変えずに、内部構造を改善すること

## コードチューニングのテクニック
## ロジック
### 答えがわかったところで評価をやめる
1. 短絡評価をつかうこと　
    - pythonの場合はand、orは短絡評価なので気にしなくても良い
2. 検索ループなどで、その値が見つかったら終わるようにする

### 頻度順の評価
- 複数のものを評価する場合、真になる確率が高いものを一番上にもってくる
- [コメント]pythonで実際にやってみたら、逆に遅くなった・・・。ちなみに、pythonにはCase文はない

In [1]:
#ISIDの社員の居住地検索のダメな例
adress_list =["東京","東京","広島","東京","東京","広島","東京","東京","海外","東京","東京","広島","東京","東京","広島","東京","東京","海外"] 

def bad_method(adress_list):
    for adress in adress_list:
        if (adress == "海外"):
            x =  1
        elif (adress == "広島"):
            x = 2
        elif (adress == "東京"):
            x =  3
            
def good_method(adress_list):
    for adress in adress_list:
        if (adress == "東京"):
            x =  1
        elif (adress == "広島"):
            x = 2
        elif (adress == "海外"):
            x =  3

In [2]:
%timeit #%timeはipythonの計測時間を計るコマンド
for i in xrange(1,10):
    bad_method(adress_list)

10000000 loops, best of 3: 17.2 ns per loop


In [3]:
%timeit #%timeはipythonの計測時間を計るコマンド
for i in xrange(1,10):
    good_method(adress_list)

The slowest run took 73.33 times longer than the fastest. This could mean that an intermediate result is being cached 
100000000 loops, best of 3: 16.3 ns per loop


### 複雑な式をテーブル参照に置き換える
- カテゴリ番号をA,B,Cに割り振る例

### ループ
### ループ外分岐
- 外に出せる分岐はループの外に出す
- [疑問]foreachがある中、いまは使わない？
- [疑問]コードがかけなかった→辞書の値として関数を代入する

In [3]:
data_list =[ 
    ["SUMTYPE_NET",1],
    ["",2],
    ["SUMTYPE_NET",3],
    ["SUMTYPE_NET",2],
    ["SUMTYPE_NET",2],
    ["",3],
    ["",2],
    ["SUMTYPE_NET",2]]

def bad():
    net_sum = 0 
    gross_sum = 0
    for i in xrange(0,len(data_list)):
        if (data_list[i][0] == "SUMTYPE_NET"):
            net_sum += data_list[i][1]
        else:
            gross_sum +=data_list[i][1]
            
##書けなかった・・・
def good():
    net_sum = 0 
    gross_sum = 0
    
    for i in xrange(0,len(data_list)):
        if (data_list[i][0] == "SUMTYPE_NET"):
            net_sum += data_list[i][1]
        else:
            gross_sum +=data_list[i][1]

In [4]:
%timeit bad()

The slowest run took 4.09 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.75 µs per loop


## ジャミング（フュージョン）
- ２つのループを組み合わせることでオーバーヘッドを減らす

In [23]:
data_list =[ 
    ["iida",100],
    ["oura",200],
    ["shibuya",300],
    ["nakane",250]]

def bad():
    for data in data_list:
        name = data_list[0]

    for data in data_list:
        score = data_list[1]
        
def good():
    for data in data_list:
        name = data_list[0]
        score = data_list[1]

In [24]:
%timeit bad()

The slowest run took 4.55 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 682 ns per loop


In [25]:
%timeit good()

The slowest run took 6.78 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 563 ns per loop


### 展開
- for文などで、１ループで扱う内容を増やすこと
- 終了条件の判断コストが大きい時（終了条件を計算してもとめるとか）は終了条件の判断回数を削減できるということでメリットがおおきい

In [26]:
data_list =[ 
    ["iida",100],
    ["oura",200],
    ["shibuya",300],
    ["nakane",250],
    ["iida",100],
    ["oura",200],
    ["shibuya",300],
    ["nakane",250]]

def good():
    sum = 0
    for i in xrange(0,8,2):
        sum +=  data_list[i][1]
        sum +=  data_list[i+1][1]
    return sum

def bad():
    sum = 0
    for i in xrange(0,8):
        sum +=  data_list[i][1]
    return sum

def normal():
    sum = 0
    for d in data_list:
        sum +=  d[1]
    return sum


In [23]:
%timeit  good()

The slowest run took 5.71 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.21 µs per loop


In [24]:
%timeit  bad()

1000000 loops, best of 3: 1.15 µs per loop


In [27]:
%timeit  normal()

The slowest run took 4.79 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 647 ns per loop


In [28]:
bad(),good(),normal()

(1700, 1700, 1700)

## ループ内の処理の最小化
- ループの外でできるものはできるだけ、ループの外で実施する

## sentinel値の利用
- forループなどを抜け出す条件が複雑な条件の場合、検索が終了したことをハンドリングするためのsentinel値を置くことで、終了判定が高速化される

## 最もbusyなループを内側に
- 内側にすることで計算量が削減できる
    - forのインデックスの初期化のコストなど

In [41]:
def good():
    x = 0
    for i in xrange(1,11):
        for j in xrange(1,10000):
            x += 1

def bad():
    x = 0
    for i in xrange(1,10000):
        for j in xrange(1,11):
            x += 1

In [42]:
%timeit  good()

100 loops, best of 3: 4.57 ms per loop


In [43]:
%timeit  bad()

100 loops, best of 3: 7.7 ms per loop


## 強度の削減
- 積算を和で表現するなど、簡単な計算でできるように変換する
- [疑問]pythonだと普通にしたほうがはやかった？

In [71]:
def bad():
    x = 2
    y = 3
    for i in xrange(1,1001):
        return x * y

def good():
    x = 2
    y = 3
    for i in xrange(1,1001):
        return x + x + x

In [72]:
%timeit  good()

The slowest run took 5.92 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 483 ns per loop


In [73]:
%timeit  bad()

The slowest run took 5.89 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 486 ns per loop


## データ変換
## 浮動小数から整数に
- 使えるところでは、浮動小数を使わず整数を使う
- [疑問]テキストではループインデックスに浮動小数でなく、整数と書いてあるが、そもそもループインデックスに浮動小数はご法度では？
- [疑問]無理やりこのテクニックを使うおうとおもうと、型変換の時間も考慮しないといけないとおもう

In [68]:
def bad():
    x = 1.0
    for i in xrange(1,1001):
        x += i
        return x

def good():
    x = 1
    for i in xrange(1,1001):
        x += i
        return x

In [69]:
%timeit  good()

The slowest run took 6.24 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 459 ns per loop


In [70]:
%timeit  bad()

The slowest run took 7.95 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 510 ns per loop


## 配列の次元数の削減
- 高次元のものを、低次元で表現可能であれば、そうする

In [93]:
#400次元を扱う
#Pythonのリストの要素は値の入れ物ではなく、ポインタとして働くので、
#n*[m*[0]]みたいな配列らしきものを作ると一箇所書き換えただけで、それを参照する部分が全部変わってしまう

def bad():
    list = [[0 for col in range(20)] for row in range(20)]

def good():
   list = [0]*400

import numpy as np
def nmpy():
   list =  np.zeros((20,20))

In [96]:
%timeit  good()

100000 loops, best of 3: 2.32 µs per loop


In [97]:
%timeit  bad()

10000 loops, best of 3: 47 µs per loop


In [98]:
%timeit  nmpy()

The slowest run took 14.74 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 809 ns per loop


## 配列参照の削減
- ループの内側で配列の参照をなるべく避ける

## 補助的なインデックスの使用
- データ型へのアクセスを高速にする方法
### 文字列の長さのインデックス
- cなどでは文字列の長さを調べるのに、文字列のの終端の0x00まで、順番にカウントしないといけない
- 文字列の長さをあらかじめ記録しておくという方法
- [疑問]c言語とか以外で意味があるのか？実際につかうか？
### 独立した並列インデックス
- [疑問]意味がわからなかった

## キャッシュの使用
- 呼び出すたびに計算するのではなく、計算結果を保持しておく

## 式
## 代表的な恒等式の利用
- 条件式を複雑なものからシンプルなものにする

In [111]:
def bad():
    x = 0
    a = [True,True,False,False,True,True,False,False,True,True,False,False]
    b = [True,False,True,False,True,False,True,False,True,False,True,False]
    for i in xrange(0,len(a)):
        if (not(a[i] or b[i])):
            x

def good():
    x = 0
    a = [True,True,False,False]
    b = [True,False,True,False]
    for i in xrange(0,len(a)):
        if (not(a[i] or b[i])):
            x

In [112]:
%timeit  good()

1000000 loops, best of 3: 1.17 µs per loop


In [113]:
%timeit  bad()

The slowest run took 4.20 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 1.87 µs per loop


## 強度削減の使用
- 掛け算を足し算に
- 指数関数を掛け算に
- 三角法を同等の手段に
- longlong型をlong型・int型に変える
- 浮動小数を固定小数・整数に変える
- シフト演算をつかう

## コンパイル時の初期化
- メソッドの引数がマジックナンバや固定値の場合、動的に変化しないため、あらかじめ計算できる

## システムルーチンへの警戒
- システムで提供されるメソッドは、過度に精度が高いものがあるので、精度が必要無い場合は無駄な計算をしている場合がある

## 正しい型の定数の使用
- 定数を宣言するとき、宣言型と同じ型を入れるようにしないといけない
- ただし、よくできたコンパイラはコンパイル時に型変換するため、パフォーマンスに影響しない

## 計算済みの結果
- 計算をキャッシュしておくことで、計算コストを下げる


## ルーチン
## ルーチンのインライン化
- 昔のプログラムでは、プログラムswap outし、ループを実行し、swap inするというペナルティが強かったらしい
- いまはあまり気にしなくてもよい

## 低水準言語への置き換え
 - pythonで書いているなら、遅い部分をCで書く

### やり方
1. アプリケーションを100%高級言語で書く
1. アプリケーションのテストを完全に行い、振る舞いを保証する
1. アプリケーションを測定して、パフォーマンス問題があるホットスポットを探る
    - プログラムの5%が実行時間の50%を占めるので特定が必要
1. その部分を低級言語で書く
